# Import libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0,'..')
import glob
#!pip install openpyxl

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import h5py
import re

from functools import partial
from multiprocessing import Pool

from lhcsmapi.Time import Time
from lhcsmapi.Timer import Timer
from scipy import signal
from lhcsmapi.Time import Time
from lhcsmapi.metadata import signal_metadata
from lhcsmapi.pyedsl.dbsignal.post_mortem.PmDbRequest import PmDbRequest
from lhcsmapi.analysis.RbCircuitQuery import RbCircuitQuery

from src.utils.mp3_excel_processing import get_fgc_timestamp, get_fgc_timestamp_missing

pd.set_option('display.max_rows', 200)
import warnings
warnings.filterwarnings('ignore')

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: htt

# Read the (clean) MP3 file

In [4]:
date_suffix = "2022_07_07"
mp3_fpa_df_raw = pd.read_excel(f"../data/RB_TC_extract_{date_suffix}.xlsx")
len(mp3_fpa_df_raw)

4773

In [5]:
# First row contains units, 9 rows contain only "Before Notebooks" and "After Notebooks" information, which need to be droped
mp3_fpa_df = mp3_fpa_df_raw.dropna(subset = ['Date (FGC)', 'Circuit Name'])
mp3_fpa_df_raw.iloc[~mp3_fpa_df_raw.index.isin(mp3_fpa_df.index)] #show droped rows

,Circuit Name,Circuit Family,Period,Date (FGC),Time (FGC),FPA Reason,Timestamp_PIC,Delta_t(FGC-PIC),Delta_t(EE_even-PIC),Delta_t(EE_odd-PIC),...,Outer cable type,I_Q_SM18,dI_Q_Acc,dI_Q_LHC,Comment,Analysis performed by,lhcsmapi version,lhcsm notebook version,Unnamed: 46,Unnamed: 47
0,NaN,RB,NaN,2008-04-01 00:00:00,13:14:15,NaN,NaN,[ms],[ms],[ms],...,NaN,[A],[A],[A],NaN,NaN,NaN,NaN,NaN,NaN
1,RB.A12,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
169,RB.A23,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
264,RB.A34,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
545,RB.A45,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
897,RB.A56,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1146,RB.A67,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1268,RB.A78,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1430,RB.A81,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1581,NaN,RB,After Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,After Notebooks,NaN,NaN,NaN


# Find FGC timestamp for each FPA event in MP3 excel

In [6]:
mp3_fpa_df['timestamp_fgc'] = mp3_fpa_df.apply(get_fgc_timestamp, axis=1)

# Find missing fgc timestamps in mp3 file

In [7]:
# some fgc timestamps have wrong hours
mp3_fpa_df_primary_missing = mp3_fpa_df[(mp3_fpa_df.timestamp_fgc.isna()) & (mp3_fpa_df['Nr in Q event'] == 1)]
mp3_fpa_df_primary_missing['timestamp_fgc'] = mp3_fpa_df_primary_missing.apply(get_fgc_timestamp_missing, axis=1)
found_fgc_timestamps_df = mp3_fpa_df_primary_missing["timestamp_fgc"].dropna()
mp3_fpa_df.loc[found_fgc_timestamps_df.index, "timestamp_fgc"] = mp3_fpa_df_primary_missing["timestamp_fgc"].dropna().values

In [8]:
print(f"{len(mp3_fpa_df)} unique fgc events") # 841 Primary quenches?
print(f"{len(mp3_fpa_df[mp3_fpa_df['Nr in Q event']==1])} mp3 primary quench entries") # 834 primary quenches with correct notes

4763 unique fgc events
986 mp3 primary quench entries


In [9]:
mp3_fpa_df.to_csv(f"../data/RB_TC_extract_{date_suffix}_processed1.csv")

# Select events to add features

In [10]:
mp3_fpa_df = pd.read_csv("../data/RB_TC_extract_2021_11_22_processed.csv")
mp3_fpa_df_unique = mp3_fpa_df.drop_duplicates(subset=['timestamp_fgc', 'Circuit Name'])

# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
lower_limit = Time.to_unix_timestamp('2014-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= lower_limit].reset_index(drop=True) 

mp3_fpa_df_to_download = mp3_fpa_df_period[mp3_fpa_df_period['Delta_t(iQPS-PIC)'].isna()]

# Add Delta_t(iQPS-PIC) timestamp

for index, row in mp3_fpa_df_to_download.iterrows():

    loc_mask = (mp3_fpa_df.timestamp_fgc == row['timestamp_fgc']) & (mp3_fpa_df["Circuit Name"] == row['Circuit Name'])

    rb_query = RbCircuitQuery(row['Circuit Family'], row['Circuit Name'], max_executions=45)
    timestamp_pic = rb_query.find_timestamp_pic(int(row['timestamp_fgc']), spark=spark)
    source_timestamp_qds_df = rb_query.find_source_timestamp_qds(int(row['timestamp_fgc']), duration=[(50, 's'), (500, 's')])

    mp3_fpa_df.loc[loc_mask, 'Delta_t(iQPS-PIC)'] = 1e-6 * (source_timestamp_qds_df['timestamp'].values - min(timestamp_pic))


mp3_fpa_df.to_csv(f"../data/RB_TC_extract_{date_suffix}_processed_filled1.csv")

# Add EE features

In [41]:
k=0
for index, row in mp3_fpa_df_to_download.iterrows():
    try:
        loc_mask = (mp3_fpa_df.timestamp_fgc == row['timestamp_fgc']) & (mp3_fpa_df["Circuit Name"] == row['Circuit Name'])

        rb_query = RbCircuitQuery(row['Circuit Family'], row['Circuit Name'], max_executions=45)

        source_timestamp_ee_even_df = rb_query.find_source_timestamp_ee(int(row['timestamp_fgc']), system='EE_EVEN')
        timestamp_ee_even = source_timestamp_ee_even_df.timestamp[0]
        u_dump_res_even_df = rb_query.query_ee_u_dump_res_pm(timestamp_ee_even, int(row['timestamp_fgc']), system='EE_EVEN',
                                            signal_names=['U_DUMP_RES'])

        source_timestamp_ee_odd_df = rb_query.find_source_timestamp_ee(int(row['timestamp_fgc']), system='EE_ODD')
        timestamp_ee_odd = source_timestamp_ee_odd_df.timestamp[0]
        u_dump_res_odd_df = rb_query.query_ee_u_dump_res_pm(timestamp_ee_odd, int(row['timestamp_fgc']), system='EE_ODD',
                                            signal_names=['U_DUMP_RES'])

        timestamp_pic = rb_query.find_timestamp_pic(int(row['timestamp_fgc']), spark=spark)

        source_timestamp_qds_df = rb_query.find_source_timestamp_qds(int(row['timestamp_fgc']), duration=[(50, 's'), (500, 's')])
        timestamp_iQps = source_timestamp_qds_df.timestamp[0]

        # sometimes values are missing in the end
        ts_iQPS_PIC = 1e-6 * (source_timestamp_qds_df['timestamp'].values - min(timestamp_pic))
        mp3_fpa_df.loc[loc_mask, 'Delta_t(iQPS-PIC)'] = np.append(ts_iQPS_PIC, np.max(ts_iQPS_PIC)*np.ones(sum(loc_mask)-len(ts_iQPS_PIC)))

        mp3_fpa_df.loc[loc_mask, 'Delta_t(EE_odd-PIC)'] = (timestamp_ee_odd - min(timestamp_pic)) / 1e6
        mp3_fpa_df.loc[loc_mask, 'Delta_t(EE_even-PIC)'] = (timestamp_ee_even - min(timestamp_pic)) / 1e6
        mp3_fpa_df.loc[loc_mask, 'U_EE_max_ODD'] = u_dump_res_odd_df.max()[0]
        mp3_fpa_df.loc[loc_mask, 'U_EE_max_EVEN'] = u_dump_res_even_df.max()[0]
    except:
        k+=1
print(k)

	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: UA23.RB.A12 from 2014-12-11 21:45:04.580 to 2014-12-11 22:01:54.580


	Querying PM event signal(s) UA23.RB.A12:U_DUMP_RES for system: QPS, className: DQAMSNRB, source: UA23.RB.A12 at 2014-12-11 21:45:15.124


	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: RR17.RB.A12 from 2014-12-11 21:45:04.580 to 2014-12-11 22:01:54.580


	Querying PM event signal(s) RR17.RB.A12:U_DUMP_RES for system: QPS, className: DQAMSNRB, source: RR17.RB.A12 at 2014-12-11 21:45:14.877


	Querying NXCALS signal(s) RB.A12.ODD:ST_ABORT_PIC, RB.A12.EVEN:ST_ABORT_PIC from 2014-12-11 21:45:13.580 to 2014-12-11 21:46:14.580


...

	Querying PM event signal(s) UA87.RB.A81:U_DUMP_RES for system: QPS, className: DQAMSNRB, source: UA87.RB.A81 at 2018-05-17 20:39:57.801


	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: RR13.RB.A81 from 2018-05-17 20:39:47.220 to 2018-05-17 20:56:37.220


	Querying PM event signal(s) RR13.RB.A81:U_DUMP_RES for system: QPS, className: DQAMSNRB, source: RR13.RB.A81 at 2018-05-17 20:39:57.300


	Querying NXCALS signal(s) RB.A81.ODD:ST_ABORT_PIC, RB.A81.EVEN:ST_ABORT_PIC from 2018-05-17 20:39:56.220 to 2018-05-17 20:40:57.220


	Querying PM event timestamps for system: QPS, className: DQAMCNMB_PMSTD, source: * from 2018-05-17 20:39:07.220 to 2018-05-17 20:48:17.220
5


In [13]:
mp3_fpa_df.to_csv(f"../data/RB_TC_extract_{date_suffix}_processed_filled1.csv")

# Reanalyze file

In [16]:
mp3_fpa_df = pd.read_csv(f"../data/RB_TC_extract_{date_suffix}_processed_filled1.csv")
# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
lower_limit = Time.to_unix_timestamp('2014-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= lower_limit].reset_index(drop=True) 

mp3_fpa_missing_values = mp3_fpa_df_period[mp3_fpa_df_period['Delta_t(iQPS-PIC)'].isna()]
len(mp3_fpa_missing_values), len(mp3_fpa_df_to_download)

(267, 267)